## Setup and Import Libraries

In [3]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [4]:
load_dotenv()

True

In [5]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = init_chat_model(model="groq:openai/gpt-oss-20b")

## Create Pinecone Vector Database

In [ ]:
pinecone = Pinecone()

index_name = "rag_index"

if not pinecone.has_index(index_name):
    pinecone.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pinecone.Index(index_name)

In [9]:
vector_store = PineconeVectorStore(
    index=index, embedding=embeddings
)

In [10]:
documents = [
    Document(
        page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
        metadata={"source": "tweet"},
    ),

    Document(
        page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
        metadata={"source": "news"},
    ),

    Document(
        page_content="Building an exciting new project with LangChain - come check it out!",
        metadata={"source": "tweet"},
    ),

    Document(
        page_content="Robbers broke into the city bank and stole $1 million in cash.",
        metadata={"source": "news"},
    ),

    Document(
        page_content="Wow! That was an amazing movie. I can't wait to see it again.",
        metadata={"source": "tweet"},
    ),

    Document(
        page_content="Is the new iPhone worth the price? Read this review to find out.",
        metadata={"source": "website"},
    ),

    Document(
        page_content="The top 10 soccer players in the world right now.",
        metadata={"source": "website"},
    ),

    Document(
        page_content="LangGraph is the best framework for building stateful, agentic applications!",
        metadata={"source": "tweet"},
    ),

    Document(
        page_content="The stock market is down 500 points today due to fears of a recession.",
        metadata={"source": "news"},
    ),

    Document(
        page_content="I have a bad feeling I am going to get deleted :(",
        metadata={"source": "tweet"},
    ),
]

documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [11]:
vector_store.add_documents(documents=documents)

['1f22cf1f-da3b-41df-8f40-f120e29e771f',
 '9cedb466-3570-4cc4-b68a-fbebbead9155',
 'dad4eedf-0b98-4f6b-803c-0d787e029168',
 'b6774d3a-3e7c-4c7a-a5a9-3af13350c800',
 '031d07aa-0e9e-43fb-8188-822caea29302',
 '550e83c0-b5e9-4036-ab6b-78c9879b457b',
 'fc070a4e-9bba-41ca-854c-86744c51cb3d',
 'd5a47628-9b9f-4c7d-b9bc-0f8fc8e4ec0e',
 '1d2b1e42-c5d9-45b0-84fa-1a6abb2cfa8c',
 'd5ebf8a5-0b21-47a1-86b3-8a14afacbf70']

## Performing Similarity

In [12]:
query = "What is the weather"

vector_store.similarity_search(query=query)

[Document(id='9cedb466-3570-4cc4-b68a-fbebbead9155', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='d5ebf8a5-0b21-47a1-86b3-8a14afacbf70', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='1d2b1e42-c5d9-45b0-84fa-1a6abb2cfa8c', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.'),
 Document(id='fc070a4e-9bba-41ca-854c-86744c51cb3d', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [14]:
query = "LangChain provides abstractions to make working with LLMs easy"

results = vector_store.similarity_search(
    query=query,
    k=2,
    filter={"source": "tweet"},
)

for result in results:
    print(f'* "{result.page_content}", metadata={result.metadata}')

* "Building an exciting new project with LangChain - come check it out!", metadata={'source': 'tweet'}
* "LangGraph is the best framework for building stateful, agentic applications!", metadata={'source': 'tweet'}


In [15]:
query = "LangChain provides abstractions to make working with LLMs easy"

results = vector_store.similarity_search_with_score(
    query=query,
    k=2,
    filter={"source": "tweet"},
)

for result, score in results:
    print(f'* [Similarity={score:.2f}] "{result.page_content}", metadata={result.metadata}')

* [Similarity=0.43] "Building an exciting new project with LangChain - come check it out!", metadata={'source': 'tweet'}
* [Similarity=0.39] "LangGraph is the best framework for building stateful, agentic applications!", metadata={'source': 'tweet'}


In [16]:
query = "Stealing from the bank is a crime"

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

retriever.invoke(input=query, filter={"source": "news"})

[Document(id='b6774d3a-3e7c-4c7a-a5a9-3af13350c800', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

## Building RAG Chain

In [17]:
system_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question. 
If you don't know the answer based on the context, say you don't know.
Provide specific details from the context to support your answer.

Context:
{context}

Question: {question}

Answer:""")

system_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. \nIf you don't know the answer based on the context, say you don't know.\nProvide specific details from the context to support your answer.\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])

In [18]:
def format_docs(documents):
    return "\n\n".join(document.page_content for document in documents)

In [19]:
rag_chain = ({ 
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | system_prompt
    | llm
    | StrOutputParser()
)

rag_chain

{
  context: VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001ADD12A2C90>, search_type='similarity_score_threshold', search_kwargs={'k': 1, 'score_threshold': 0.5})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. \nIf you don't know the answer based on the context, say you don't know.\nProvide specific details from the context to support your answer.\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ADCF966480>, async_client=<groq.resources.chat.c

In [20]:
query = "Stealing from the bank is a crime"

response= rag_chain.invoke(query)
response

'Yes. According to the context, robbers broke into the city bank and stole $1\u202fmillion in cash. This act of taking the bank’s money without permission constitutes theft (robbery), which is a criminal offense.'

In [21]:
def query_rag(question):
    print(f"Question: {question}")
    print("-" * 50)
    
    answer = rag_chain.invoke(question)
    print(f"Answer: {answer}")
    
    # Get source documents separately if needed
    docs = retriever.get_relevant_documents(question)
    print("\nSource Documents:")
    for i, doc in enumerate(docs):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200] + "...")

In [22]:
question = "Stealing from the bank is a crime"

query_rag(question=question)

Question: Stealing from the bank is a crime
--------------------------------------------------
Answer: Yes. The context states that robbers broke into the city bank and stole $1 million in cash, which is an illegal act.

Source Documents:

--- Source 1 ---
Robbers broke into the city bank and stole $1 million in cash....
